# gurobipy 8.0 breaks back compatibility with float("inf")

The `gurobipy` package has broken long standing `inf`  functionality with their 8.0 release. Historically (dating back at least 5 years) `inf` is replaced with `GRB.INFINITY` during model building.  The 8.0 release instead raises an exception upon encountering `inf`. (But only with constraint building - `inf` can still be used to represent unboundedness with variable creation).

To demonstrate this functionality, we will use a netflow.xlsx file that has infinity as part of the input data. Specifically, this file specifies an unbounded upper limit for product flow between Detroit and Seattle. Other than this one edit, the data is consistent with Gurobi netflow example [here](http://www.gurobi.com/documentation/7.5/examples/netflow_py.html). Since there is an optimal solution with slack on the Detroit to Seattle constraint, this edit doesn't change the optimization result.

You can download this netflow.xlsx file from [here](https://drive.google.com/drive/folders/18w1hwJDrh4RJWwJi2tZb3qVUisUT5Q2k?usp=sharing). 

In [1]:
import os
assert os.path.isfile("netflow.xlsx"), "need the data file"

Now let's read this data into `pandas.DataFrame` objects. [pandas](https://pandas.pydata.org/) is a wildly popular data science package. We will use the standard `pandas` Excel reader to convert each data sheet into a `DataFrame`.

In [2]:
import pandas as pd
commodities_df = pd.read_excel("netflow.xlsx", "Commodities")
nodes_df = pd.read_excel("netflow.xlsx", "Nodes")
arcs_df = pd.read_excel("netflow.xlsx", "Arcs")
cost_df = pd.read_excel("netflow.xlsx", "Cost")
inflow_df = pd.read_excel("netflow.xlsx", "Inflow")

Now let's convert these `DataFrame` objects into the data structures expected by the netflow.py code.

In [3]:
from gurobipy import *
commodities = list(commodities_df["Name"])
nodes = list(nodes_df["Name"])
arcs, capacity = multidict(arcs_df.set_index(['Source', 'Destination']).
                              to_dict()["Capacity"])
cost = cost_df.set_index(["Commodity", "Source", "Destination"]).to_dict()["Cost"]
inflow = inflow_df.set_index(["Commodity", "Node"]).to_dict()["Quantity"]

As you can see, `pandas` has created an `inf` value as part of normal data reading. More on this later.

In [4]:
capacity

{(u'Denver', u'Boston'): 120.0,
 (u'Denver', u'New York'): 120.0,
 (u'Denver', u'Seattle'): 120.0,
 (u'Detroit', u'Boston'): 100.0,
 (u'Detroit', u'New York'): 80.0,
 (u'Detroit', u'Seattle'): inf}

The large-ish code block below is an exact copy of the solving section of [netflow.py](http://www.gurobi.com/documentation/7.5/examples/netflow_py.html).
For Gurobi 7.5 this model solves and returns an optimal solution of 5,500. I believe it will run on versions of Gurobi as far back as 5.6.2 so long as you edit it to use `addVar`, `addConstr` instead of `addVars` and `addConstrs`.

If you run this with Gurobi 8.0 it throws an exception. Feel free to rerun the notebook on your own installation.

In [5]:
GRB.VERSION_MAJOR, GRB.VERSION_MINOR, GRB.VERSION_TECHNICAL

(7, 5, 2)

In [6]:
# Create optimization model
m = Model('netflow')

# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

# Arc capacity constraints
m.addConstrs(
    (flow.sum('*',i,j) <= capacity[i,j] for i,j in arcs), "cap")

# Equivalent version using Python looping
# for i,j in arcs:
#   m.addConstr(sum(flow[h,i,j] for h in commodities) <= capacity[i,j],
#               "cap[%s,%s]" % (i, j))


# Flow conservation constraints
m.addConstrs(
    (flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*')
    for h in commodities for j in nodes), "node")
# Alternate version:
# m.addConstrs(
#   (quicksum(flow[h,i,j] for i,j in arcs.select('*',j)) + inflow[h,j] ==
#     quicksum(flow[h,j,k] for j,k in arcs.select(j,'*'))
#     for h in commodities for j in nodes), "node")

# Compute optimal solution
m.optimize()

Optimize a model with 16 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  5.500000000e+03


This example involves reading data from an .xlsx file with the `pandas` package. The same sort of problem can be demonstrated using `pandas` to read from `.csv` or SQLite files (and likely from nearly every file format `pandas` supports - see Jeff Reback's comment re: infinity support [here](https://github.com/pandas-dev/pandas/issues/10065)).

It's not just `pandas` that likes to introduce `inf`. `xlrd` can do it with .xls files, for [example](http://www.gurobi.com/documentation/7.5/examples/diet4_py.html).

Note that you can't determine if your engine is at risk of this exception failure simply by scanning your code for explicit references to `float("inf")`. Since `inf` is a fundamental part of Python, any of the packages you are importing might themselves be introducing `inf` just as `pandas` did in this example. I am not aware of a turn-key work around for existing code to avoid this problem. 

Note that there isn't any backwards-compatibility breakage with using `inf` to represent unboundedness when adding variables. `gurobipy` will still quietly replace `inf` with `GRB.INFINITY` when calling `addVar`. It appears to be only the constraint building code that has changed.